기본 세팅 및 install

In [ ]:

!pip install -q pytorch-lightning==1.9.4
!pip install -q transformers
!pip install sentencepiece
!pip install boto3


from google.colab import drive
# drive.mount('/content/drive')
from __future__ import print_function
import time
import boto3
from random import random


aws_access_key_id = '',
aws_secret_access_key = ''
s3 = boto3.client(
    's3',
    aws_access_key_id="본인의 키 넣기",
    aws_secret_access_key="본인의 키 넣기"
)

# 인퍼런스를 위한 import

import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import pytorch_lightning as pl
import transformers
import os
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AdamW
# transformers.__version__
import boto3

def s3_connection():
    try:
        # s3 클라이언트 생성
        s3 = boto3.client(
            service_name="s3",
            region_name="ap-northeast-2",
            aws_access_key_id="본인의 키 넣기",
            aws_secret_access_key="본인의 키 넣기",
        )
    except Exception as e:
        print(e)
    else:
        print("s3 bucket connected!")
        return s3

s3 = s3_connection()

my_config = {
    'project_name': 'voice_pp',
    'model_name': 'microsoft/deberta-v3-large',
    'num_workers': 0,  # adjust as needed
    'num_labels': 2,
    'max_epochs': 20,
    'patience' : 2,
    'max_length': 512,
    'batch_size': 3,
    'learning_rate':6e-6,
    'accumulate_grad_batches': 4,
    'warmup_steps': None,  # to be set later
    'wandb_api_key': '본인의 키 넣기',
    'data_dir': '/content/drive/MyDrive/이브와 모델 시도/data_dir',
    'result_dir': '/content/drive/MyDrive/이브와 모델 시도/result_dir'
}

class NewsClassifier(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.model = AutoModelForSequenceClassification.from_pretrained(config['model_name'], num_labels=config['num_labels'])

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids=input_ids, attention_mask=attention_mask)

    def training_step(self, batch, batch_idx):
        outputs = self(batch['input_ids'], batch['attention_mask'], batch['labels'])
        loss = outputs.loss
        self.log('train_loss',loss)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        outputs = self(batch['input_ids'], batch['attention_mask'],  batch['labels'])
        loss = outputs.loss
        logits = outputs.logits.argmax(dim=1)
        f1 = f1_score(batch['labels'].cpu(), logits.cpu(), average='macro')
        self.log('val_loss',loss)
        self.log('val_f1_score',f1)
        return {'val_loss': loss, 'val_f1_score': f1}

    def test_step(self, batch, batch_idx):
        outputs = self(batch['input_ids'], batch['attention_mask'])
        logits = outputs.logits.argmax(dim=1)
        return {'id': batch['input_ids'], 'label': logits}

    def test_epoch_end(self, outputs):
        ids = []
        labels = []
        for out in outputs:
            ids.extend(out['id'])
            labels.extend(out['label'].cpu().tolist())  # Move the label tensor to CPU
        submission = pd.DataFrame({'id': ids, 'label': labels})
        submission.to_csv(os.path.join(self.config['result_dir'], 'submission_debertav3.csv'), index=False)

        return submission

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.config['learning_rate'])
        return optimizer


tokenizer = AutoTokenizer.from_pretrained(my_config['model_name'])
model = NewsClassifier.load_from_checkpoint('/content/drive/MyDrive/이브와 모델 시도/result_dir/best_model-v1.ckpt',config=my_config)
def preprocess_text(text):
    encoded_text = tokenizer(text, truncation=True, padding='max_length', max_length=my_config['max_length'], return_tensors='pt')
    return encoded_text

def classify_text(text):
    model.eval()
    with torch.no_grad():
        input_data = preprocess_text(text)
        input_ids = input_data['input_ids']
        attention_mask = input_data['attention_mask']

        outputs = model(input_ids, attention_mask=attention_mask)  # Remove token_type_ids argument
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_label = torch.argmax(probabilities, dim=1).item()
    return predicted_label, probabilities

In [ ]:
# 정상인 경우
# f = open('/content/output.txt','r')
# data = f.read().splitlines()
# new_text = data[0]

# 피싱인 경우,
import json
with open ("asrOutput (4).json", "r") as f:
    data = json.load(f)
new_text = data["results"]['transcripts'][0]['transcript']


predicted_label, probabilities = classify_text(new_text)
print("Predicted Label:", predicted_label)
print("Label Probabilities:", probabilities)

f = open("file.txt",'w')
if predicted_label ==1:
  f.write(str(predicted_label)+"\n")
else:
  f.write(str(predicted_label)+"\n")

f.close()

def s3_connection():
    try:
        # s3 클라이언트 생성
        s3 = boto3.client(
            service_name="s3",
            region_name="ap-northeast-2",
            aws_access_key_id="본인의 키 넣기",
            aws_secret_access_key="본인의 키 넣기",
        )
        s3.upload_file("file.txt","raspberrypi.transfer","label.txt")
    except Exception as e:
        print(e)
    else:
        print("s3 bucket connected!")
        return s3

s3 = s3_connection()

이후 버킷 관련 (업로드 코드)

In [ ]:
# 버킷올리기

aws_access_key_id = '본인의 키 넣기'
aws_secret_access_key = '본인의 키 넣기'

# AWS 클라이언트 생성
s3_client = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

import boto3

def upload_file_to_s3(local_file_path, bucket_name, s3_file_path, aws_access_key_id, aws_secret_access_key):
    """
    로컬 파일을 S3 버킷에 업로드하는 함수
    :param local_file_path: 업로드할 로컬 파일의 경로 (예: "C:\\Users\\USER\\Desktop\\1234.mp3")
    :param bucket_name: 업로드할 S3 버킷의 이름
    :param s3_file_path: S3 버킷 내에서 파일이 저장될 경로 및 이름 (예: "uploads/1234.mp3")
    :param aws_access_key_id: AWS 액세스 키 ID
    :param aws_secret_access_key: AWS 비밀 액세스 키
    """
    try:
        s3_client = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
        s3_client.upload_file(local_file_path, bucket_name, s3_file_path)
        print(f"파일이 성공적으로 S3 버킷에 업로드되었습니다. 버킷: '{bucket_name}', 경로: '{s3_file_path}'")
    except Exception as e:
        print(f"파일 업로드에 실패했습니다. 오류 메시지: {str(e)}")

# 업로드할 로컬 파일의 경로
local_file_path = "/content/example.mp3"

# S3 버킷의 이름
bucket_name = "giveittome"
s3_file_path = "hello/label00.mp3"
aws_access_key_id = '본인의 키 넣기'
aws_secret_access_key = '본인의 키 넣기'

upload_file_to_s3(local_file_path, bucket_name, s3_file_path, aws_access_key_id, aws_secret_access_key)

import time

transcribe = boto3.client(
    "transcribe",
    region_name="ap-northeast-2",
    aws_access_key_id='본인의 키 넣기',
    aws_secret_access_key='본인의 키 넣기',
)

url_lst = []

transcribe = boto3.client("transcribe",region_name="ap-northeast-2")
job_name = f"transcribe_test"
job_url = f"s3://giveittome/hello/label00.mp3"
# transcribe.start_transcription_job(
#     TranscriptionJobName = job_name,
#     Media = {"MediaFileUri": job_url},
#     MediaFormat = "mp3",
#     LanguageCode = 'ko-KR',
# )

# while True:
#     status = transcribe.get_transcription_job(TranscriptionJobName = job_name)
#     if status["TranscriptionJob"]["TranscriptionJobStatus"] in ['COMPLETED','FAILED']:
#         break
#     print("Not ready yet...")
#     time.sleep(5)
# print(status)